# Sampling From Line Graph Distribution

In [2]:
import numpy as np
import torch
import math

In [3]:
#arbitrary non-linear latent model function
def f(x):
    
    return x**3 - 2*x**2 + 1

In [4]:
def sample_full_rank_matrix(n=3, lower=-10, upper=10):
    while True:
        matrix = torch.FloatTensor(n, n).uniform_(lower, upper)
        if torch.det(matrix).item() != 0:
            return matrix
        
# def sample_full_rank_matrix(num_latent=3, num_observed=3, lower=-10, upper=10):
#     rank = min(num_latent, num_observed)
#     matrix = torch.zeros((num_observed, num_latent))
#     for i in range(rank):
#         random_vector = torch.rand(num_latent if num_observed >= num_latent else num_observed) * (upper - lower) + lower
#         if num_observed >= num_latent:
#             matrix[i, :] = random_vector
#         else:
#             matrix[:, i] = random_vector
    
#     # If num_observed < num_latent, we need to ensure the columns are linearly independent
#     if num_observed < num_latent:
#         # Perform QR decomposition to orthogonalize the columns
#         q, r = torch.linalg.qr(matrix)
#         # Use Q (which has orthogonal columns) and scale it to the range
#         scale_factor = (upper - lower) / 2.0
#         matrix = q[:, :num_latent] * scale_factor + scale_factor + lower

#     return matrix

In [5]:
def sample_full_rank_matrix_sparse(n=3, lower=-10, upper=10):
    matrix = torch.FloatTensor(n, n).uniform_(lower, upper)
    # Set the diagonal elements to random values to ensure full rank
    diag = torch.FloatTensor(n).uniform_(lower, upper)
    matrix.fill_diagonal_(0)  # Set diagonal to zero before filling with random values
    matrix += torch.diag(diag)
    
    # Get the indices of the off-diagonal elements
    triu_indices = torch.triu_indices(n, n, offset=1)
    tril_indices = torch.tril_indices(n, n, offset=-1)
    off_diagonal_indices = torch.cat((triu_indices, tril_indices), dim=1)
    
    # Randomly select half of the off-diagonal elements to set to zero
    num_off_diagonal_elements = off_diagonal_indices.shape[1]
#     num_elements_to_zero = num_zero
    num_elements_to_zero = torch.randint(0, n**2-n+1, (1,)).item()
#     print(num_elements_to_zero)
    
    # Generate random indices to zero out without replacement
    indices_to_zero = torch.randperm(num_off_diagonal_elements)[:num_elements_to_zero]
    
    # Zero out the selected off-diagonal elements
    matrix[off_diagonal_indices[0, indices_to_zero], off_diagonal_indices[1, indices_to_zero]] = 0.0
    
    return matrix

In [6]:
def sample(G, f, variances):
    noise = [torch.normal(0, torch.sqrt(variance)).requires_grad_() for variance in variances]
    U = [noise[0]]
    for i in range(1, len(variances)):
        U.append(f(U[i-1]) + noise[i])
    U_tensor = torch.stack(U)  # Convert list of tensors to a tensor
    X = torch.matmul(G, U_tensor.unsqueeze(1)).squeeze(1)  # Matrix multiplication and remove extra dimension
    return U_tensor, X

In [7]:
def sample_n(G, f, variances, n):
    U_list, X_list = [], []
    for _ in range(n):
        U, X = sample(G, f, variances)
        U_list.append(U)
        X_list.append(X)
    U_stacked = torch.stack(U_list)
    X_stacked = torch.stack(X_list)
    return U_stacked, X_stacked

In [8]:
variances = torch.rand(4)*5
G = sample_full_rank_matrix(4)
G_hat = sample_full_rank_matrix()
U, X = sample(G, f, variances)
U,X

(tensor([-2.6052e+00, -3.0404e+01, -2.9952e+04, -2.6873e+13],
        grad_fn=<StackBackward0>),
 tensor([ 1.3709e+14,  2.9185e+13, -8.6944e+13, -7.8140e+12],
        grad_fn=<SqueezeBackward1>))

In [9]:
# for _ in range(50):
#     X = sample_n(G,f,variances,2000)
#     X = torch.tensor(X)
#     print(compute_top_order(X, eta_G=0.001, eta_H=0.001, normalize_var=False, dispersion="var"))

In [10]:
sample_n(G, f, variances, 100)[0].shape

torch.Size([100, 4])

# Calculating Jacobian

In [25]:
def gaussian_pdf(x, mu, var):
#     print(x,mu,var)
    return (1/torch.sqrt(2*torch.pi*var))*torch.exp(-0.5*((x-mu)**2/var))

In [26]:
def p_u(U, variances):
    # Calculate the product of Gaussian PDFs
    prod = gaussian_pdf(U[0], torch.tensor(0.0), variances[0])
    for i in range(1, len(U)):
        prod *= gaussian_pdf(U[i], f(U[i-1]), variances[i])
    return prod

In [27]:
def log_pu(U, variances):
    return torch.log(p_u(U, variances))

In [28]:
def score_u(U, variances):
    return torch.autograd.grad(outputs=log_pu(U, variances), inputs=U, create_graph=True)[0]

In [29]:
def jacobian_score_u(U, variances):
    grad_log_pu = score_u(U, variances)
    hessian = torch.zeros(U.size(0), U.size(0))
    for i in range(U.size(0)):
        grad_grad_log_pu = torch.autograd.grad(outputs=grad_log_pu[i], inputs=U, create_graph=True)
        hessian[i] = grad_grad_log_pu[0]
    return hessian

In [30]:
def estimate_jacobian_score_u(G, G_hat, X, U, variances):
    beta = torch.matmul(torch.inverse(G), G_hat)
    return torch.matmul(torch.matmul(beta, jacobian_score_u(U, variances)), beta.T)

In [31]:
def diag(mat):
    return torch.diag(mat)

In [32]:
# Not needed ---------------------

In [33]:
def p_x(X):
    return p_u(torch.matmul(torch.inverse(G), X.unsqueeze(1)))*torch.abs(torch.det(torch.inverse(G)))

In [34]:
def log_px(X):
    return torch.log(p_x(X))

In [35]:
def score_x(X):
    return torch.autograd.grad(outputs=log_px(X), inputs=X, create_graph=True)[0]

In [36]:
def jacobian_score_x(X):
    grad_log_px = score_x(X)
    hessian = torch.zeros(X.size(0), X.size(0))
    for i in range(X.size(0)):
        grad_grad_log_px = torch.autograd.grad(outputs=grad_log_px[i], inputs=X, create_graph=True)
        hessian[i] = grad_grad_log_px[0]
    return hessian
    

In [37]:
# ---------------------

In [38]:
variances = torch.rand(3)*5
G = sample_full_rank_matrix()
G_hat = sample_full_rank_matrix()
U, X = sample(G, f, variances)
U,X

(tensor([0.6207, 0.6440, 1.4228], grad_fn=<StackBackward0>),
 tensor([-1.5762, 16.5409,  9.4130], grad_fn=<SqueezeBackward1>))

In [39]:
jacobian_score_u(U, variances)

tensor([[-3.1885e+00, -2.1120e+00,  0.0000e+00],
        [-2.1120e+00, -2.8368e+00, -8.6922e-01],
        [ 2.7501e-08, -8.6922e-01, -6.5267e-01]], grad_fn=<CopySlices>)

In [40]:
jacobian_score_x(X)

TypeError: p_u() missing 1 required positional argument: 'variances'

In [ ]:
diag(jacobian_score_u(U, variances))

tensor([-2.4205e+02, -1.1429e+07, -5.4575e+00], grad_fn=<DiagBackward0>)

In [25]:
estimate_jacobian_score_u(G, G_hat, X, U, variances)

NameError: name 'gaussian_pdf' is not defined

In [26]:
diag(estimate_jacobian_score_u(G, G_hat, X, U, variances))

NameError: name 'gaussian_pdf' is not defined

In [27]:
score_x(X)

TypeError: p_u() missing 1 required positional argument: 'variances'

In [ ]:
score_u(U)

tensor([ 0.0886,  1.3798, -0.3731], grad_fn=<AddBackward0>)

In [28]:
torch.matmul(G.T, score_x(X).unsqueeze(1))

TypeError: p_u() missing 1 required positional argument: 'variances'

# Variance Analysis

In [1178]:
variances = torch.rand(3)*5
G = sample_full_rank_matrix()
G_hat = sample_full_rank_matrix()
U, X = sample_n(G, f, variances, 100)
U,X

(tensor([[-1.2816e+00,  5.4852e-01,  3.8196e-01],
         [ 2.7925e-01, -1.4751e-01,  4.1896e-02],
         [-1.2223e+00, -1.8207e-01,  6.6510e-02],
         [-1.2953e+00,  3.1872e+00,  1.0144e+01],
         [-2.8116e-01, -1.2051e+00,  1.1980e+00],
         [-8.2955e-01,  2.0070e+00,  4.0439e+00],
         [-2.4473e+00,  7.3781e+00,  5.4387e+01],
         [-4.5617e-01,  3.3566e+00,  1.1190e+01],
         [-3.6973e+00,  1.6008e+01,  2.5632e+02],
         [-1.4092e-01,  2.2534e-01, -1.4295e-01],
         [ 3.1128e+00,  1.1077e+01,  1.2266e+02],
         [ 7.8855e-01, -1.6983e-01, -1.8168e-01],
         [ 3.0941e+00,  9.6191e+00,  9.2449e+01],
         [-1.0631e-01, -1.4415e-01,  1.2212e-01],
         [ 1.6246e+00,  4.8165e+00,  2.3661e+01],
         [-6.8146e-02, -3.9815e-01,  1.6055e-02],
         [-8.1533e-01,  3.8914e-01,  8.3463e-02],
         [-7.7810e-01,  1.4316e+00,  2.2858e+00],
         [-1.7389e+00,  3.6894e+00,  1.3373e+01],
         [ 1.6970e-01,  5.6641e-01,  4.2427e-01],


In [1179]:
true_diag_jacobians = torch.zeros_like(U)
estimated_diag_jacobians = torch.zeros_like(U)

In [1180]:
for i in range(len(U)):
    true_diag_jacobians[i] = diag(jacobian_score_u(U[i], variances))
    estimated_diag_jacobians[i] = diag(estimate_jacobian_score_u(G, G_hat, X[i], U[i], variances))

In [1181]:
torch.var(true_diag_jacobians, dim=0, unbiased=True)

tensor([4.0091e+02, 5.3193e+08, 7.1547e-11], grad_fn=<VarBackward0>)

In [1182]:
torch.var(estimated_diag_jacobians, dim=0, unbiased=True)

tensor([1.8752e+10, 3.8981e+11, 3.2277e+11], grad_fn=<VarBackward0>)

# Full Simulation

In [1183]:
#adjustable params
num_latent = 4
num_samples = 500
variance_max = 1
lower_G = -1 #min possibel value in G or G_hat
upper_G = 1 #max possible value in G or G_hat
f = lambda x:  x**2

In [1184]:
#unadjustable params (for now)
num_observed = num_latent

In [1198]:
def simulate(num_latent, num_observed, num_samples, variance_max, lower_G, upper_G, f):
    variances = torch.rand(num_latent)*variance_max
#     G = sample_full_rank_matrix(num_latent, lower_G, upper_G)
    G = sample_full_rank_matrix_sparse(num_latent, lower_G, upper_G)
#     G_hat = sample_full_rank_matrix(num_latent, lower_G, upper_G)
#     G_hat = G - sample_full_rank_matrix(num_latent, 0, 0.01)
#     G_hat = torch.eye(num_latent)
    G_hat = sample_full_rank_matrix_sparse(num_latent, lower_G, upper_G)
    b = torch.matmul(torch.inverse(G), G_hat)
    print(b)
    U, X = sample_n(G, f, variances, num_samples)
    
    true_diag_jacobians = torch.zeros_like(U)
    estimated_diag_jacobians = torch.zeros_like(U)
    
    for i in range(len(U)):
        true_diag_jacobians[i] = diag(jacobian_score_u(U[i], variances))
        estimated_diag_jacobians[i] = diag(estimate_jacobian_score_u(G, G_hat, X[i], U[i], variances))
    
    print('True Variance:', torch.var(true_diag_jacobians, dim=0, unbiased=True).tolist())
    print('Estimated Variance:', torch.var(estimated_diag_jacobians, dim=0, unbiased=True).tolist())
    print()
    passed=False
    for var in torch.var(true_diag_jacobians, dim=0, unbiased=True).tolist():
        if np.isclose(var, 0.0, 1e-5):
            passed=True
    for var in torch.var(estimated_diag_jacobians, dim=0, unbiased=True).tolist():
        if np.isclose(var, 0.0, 1e-5):
            passed=False
    if passed:
        print('Passed identifiabability')
    else:
        print('Failed identifiabability')

In [1208]:
simulate(num_latent, num_observed, num_samples, variance_max, lower_G, upper_G, f)

tensor([[ 1.1331,  0.0000, -5.1714,  0.0000],
        [-0.1646, -0.3573,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -2.6082,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -4.8368]])
True Variance: [28.008695602416992, 805.4207153320312, 308027.4375, 5.715526251066916e-14]
Estimated Variance: [220382656.0, 13.891548156738281, 14254862.0, 3.133234297014731e-11]

Failed identifiabability


# Estimating Jacobian

In [1168]:
def Stein_grad(X, s, eta):
    """
    Estimates the score of X at the provided samples points, using Stein identities
    """
    n, d = X.shape

    X_diff = X.unsqueeze(1)-X.unsqueeze(0)
    K = torch.exp(-torch.norm(X_diff, dim=2, p=2)**2 / (2 * s**2)) / s

    nablaK = -torch.einsum('kij,ik->kj', X_diff, K) / s**2
    return torch.matmul(torch.inverse(K + eta * torch.eye(n)), nablaK)

In [70]:
def median_pairwise(X):
    X_diff = X.unsqueeze(1) - X.unsqueeze(0)
    pairwise_distances = torch.norm(X_diff, dim=2, p=2)
    
    upper_triangular_values = pairwise_distances[torch.triu(torch.ones_like(pairwise_distances), diagonal=1) == 1]
    return upper_triangular_values.median().item()

In [71]:
s = median_pairwise(X)
eta = 0.001

G_stein = Stein_grad(X,s,eta)


In [72]:
x = X[0]
y = G_stein[0]

In [73]:
J = torch.zeros_like(y.unsqueeze(1).mm(X[0].unsqueeze(0)))

In [74]:
y[0].backward(retain_graph=True)

In [75]:
for i in range(len(y)):
    # Zero gradients (important to clear previous gradients)
    if x.grad is not None:
        x.grad.zero_()
    
    # Compute the gradient of the i-th function with respect to x
    y[i].backward(retain_graph=True)
    
    # Copy the gradients (which are the partial derivatives)
    Jacobian[i] = x.grad.clone()

print(Jacobian)

AttributeError: 'NoneType' object has no attribute 'clone'

In [58]:
import torch

# Define your functions f1, f2, and f3
def f1(x):
    # Placeholder for actual function
    return x[0] ** 2 + x[1] - x[2]  # Example function of x

def f2(x):
    # Placeholder for actual function
    return x[1] ** 2 + x[2] - x[0]  # Example function of x

def f3(x):
    # Placeholder for actual function
    return x[2] ** 2 + x[0] - x[1]  # Example function of x

# Define the vector-valued function f(x)
def f(x):
    return torch.tensor([f1(x), f2(x), f3(x)], dtype=torch.float32)

# Define the point x at which you want to evaluate the Jacobian
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# Compute the function value at x
y = f(x)

# Initialize a Jacobian matrix with zeros
Jacobian = torch.zeros((3, 3))

# Compute the Jacobian matrix
for i in range(3):
    # Zero gradients (important to clear previous gradients)
    if x.grad is not None:
        x.grad.zero_()
    
    # Compute the gradient of the i-th function with respect to x
    y[i].backward(retain_graph=True)
    
    # Copy the gradients (which are the partial derivatives)
    Jacobian[i] = x.grad.clone()

print(Jacobian)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [14]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


In [15]:
out = (inp+1).pow(2).t()

In [16]:
out.backward(torch.ones_like(out), retain_graph = True)

In [ ]:
def Stein_grad(X, s, eta): # Not used
    n, d = X.shape

    X_diff = X.unsqueeze(1)-X
    K = torch.exp(-torch.norm(X_diff, dim=2, p=2)**2 / (2 * s**2)) / s

    nablaK = -torch.einsum('kij,ik->kj', X_diff, K) / s**2
    return torch.matmul(torch.inverse(K + eta * torch.eye(n)), nablaK)

In [45]:
def Stein_hess(X, eta_G, eta_H, s = None):
    """
    Estimates the diagonal of the Hessian of log p_X at the provided samples points
    X, using first and second-order Stein identities
    """
    n, d = X.shape
    
    X_diff = X.unsqueeze(1)-X
    if s is None:
        D = torch.norm(X_diff, dim=2, p=2)
        s = D.flatten().median()
    K = torch.exp(-torch.norm(X_diff, dim=2, p=2)**2 / (2 * s**2)) / s
    
    nablaK = -torch.einsum('kij,ik->kj', X_diff, K) / s**2
    G = torch.matmul(torch.inverse(K + eta_G * torch.eye(n)), nablaK)
    
    nabla2K = torch.einsum('kij,ik->kj', -1/s**2 + X_diff**2/s**4, K)
    return -G**2 + torch.matmul(torch.inverse(K + eta_H * torch.eye(n)), nabla2K)

In [46]:
def compute_top_order(X, eta_G, eta_H, normalize_var=True, dispersion="var"):
    n, d = X.shape
    order = []
    active_nodes = list(range(d))
    for i in range(d-1):
        H = Stein_hess(X, eta_G, eta_H)
        if normalize_var:
            H = H / H.mean(axis=0)
        if dispersion == "var": # The one mentioned in the paper
            l = int(H.var(axis=0).argmin())
        elif dispersion == "median":
            med = H.median(axis = 0)[0]
            l = int((H - med).abs().mean(axis=0).argmin())
        else:
            raise Exception("Unknown dispersion criterion")
        order.append(active_nodes[l])
        active_nodes.pop(l)
        X = torch.hstack([X[:,0:l], X[:,l+1:]])
    order.append(active_nodes[0])
    order.reverse()
    return order
    